In [2]:
import os
import pandas as pd
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 各種パスの設定
######################################################################
base_path = '' # ベースとなるパスを指定してください。#######
######################################################################
os.makedirs(os.path.join(base_path,'model'), exist_ok=True)  # 学習済みモデルの保存するディレクトリを作成
os.makedirs(os.path.join(base_path,'output'), exist_ok=True)  # 提出用ファイルを出力するディレクトリを作成
train_data_path = os.path.join(base_path,'data/train_data.csv') # 訓練データのパスを指定
test_data_path = os.path.join(base_path,'data/test_data.csv') # テストデータのパスを指定
submit_data_path = os.path.join(base_path,'data/submission.csv') # 提出用サンプルfileのパスを指定
model_file_path = os.path.join(base_path,'model/best_model.pt') # 学習済みモデルのパスを指定
output_file_path = os.path.join(base_path,'output/20230303_submission.csv') # 提出用ファイルのパスを指定

In [3]:
#　データセットを作成するクラスを定義します。
class PaperDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        title = self.data.iloc[index]['title']
        abstract = self.data.iloc[index]['abstract']
        text = title + ' ' + abstract
        labels = self.data.iloc[index]['y']
        
        # inputsの大きさをmodelが扱えるmax_length(512)に指定
        inputs = self.tokenizer(text, max_length=512, truncation=True)
       
        # print(len(inputs['input_ids']))
        
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'targets': torch.tensor(labels, dtype=torch.float)
        }

In [4]:
# datasetで定義されたバッチ形式の__getitem__を処理して、オリジナルのtorch.Tensorにする関数
def collate_fn(batch):
    input_ids = pad_sequence([torch.tensor(item["input_ids"]) for item in batch], batch_first=True)
    attention_mask = pad_sequence([torch.tensor(item["attention_mask"]) for item in batch], batch_first=True)
    # labels_0 = ([item['targets'][0] for item in batch])
    # labels_1 = ([item['targets'][1] for item in batch])
    # labels = torch.tensor([labels_0, labels_1])
    labels = torch.tensor([item['targets'] for item in batch])


    return {"input_ids": input_ids, "attention_mask": attention_mask, 'targets': labels}


In [5]:
# 各種データの読込
df = pd.read_csv(train_data_path) # 訓練データの読込
test_df = pd.read_csv(test_data_path) # テストデータの読込
test_df['y'] = 0 # テストデータのＹの値を初期化
submit_df = pd.read_csv(submit_data_path) # 提出用散布リファイルの読込

In [6]:
df

,id,title,year,abstract,keywords,y
0,1,Hierarchical Adversarially Learned Inference,2018,We propose a novel hierarchical generative mod...,"generative, hierarchical, unsupervised, semisu...",0
1,2,Learning to Compute Word Embeddings On the Fly,2018,Words in natural language follow a Zipfian dis...,"NLU, word embeddings, representation learning",0
2,3,Graph2Seq: Scalable Learning Dynamics for Graphs,2018,Neural networks are increasingly used as a gen...,NaN,0
3,4,Generating Differentially Private Datasets Usi...,2018,"In this paper, we present a technique for gene...","generative adversarial networks, differential ...",0
4,5,Representing dynamically: An active process fo...,2018,We propose an unsupervised method for building...,"Generative Models, Latent representations, Pre...",0
...,...,...,...,...,...,...
4969,4970,Convolutional Conditional Neural Processes,2020,We introduce the Convolutional Conditional Neu...,"Neural Processes, Deep Sets, Translation Equiv...",1
4970,4971,Gradient Descent Maximizes the Margin of Homog...,2020,"In this paper, we study the implicit regulariz...","margin, homogeneous, gradient descent",1
4971,4972,Adversarial Training and Provable Defenses: Br...,2020,"We present COLT, a new method to train neural ...","adversarial examples, adversarial training, pr...",1
4972,4973,Differentiable Reasoning over a Virtual Knowle...,2020,We consider the task of answering complex mult...,"Question Answering, Multi-Hop QA, Deep Learnin...",1


In [7]:
test_df

,id,title,year,abstract,keywords
0,1,StyleAlign: Analysis and Applications of Align...,2022,"In this paper, we perform an in-depth study of...","StyleGAN, transfer learning, fine tuning, mode..."
1,2,Embedding a random graph via GNN: mean-field i...,2021,We develop a theory for embedding a random gra...,"Graph neural network, graph embedding, multi-r..."
2,3,BBRefinement: an universal scheme to improve p...,2021,We present a conceptually simple yet powerful ...,"object detection, deep neural networks, refine..."
3,4,Assessing Deep Reinforcement Learning Policies...,2022,Deep reinforcement learning algorithms have re...,NaN
4,5,Few-shot Learning with Big Prototypes,2022,"Using dense vectors, i.e., prototypes, to repr...","Prototype, Few-shot Learning, Meta-learning"
...,...,...,...,...,...
6388,6389,Transferable Feature Learning on Graphs Across...,2021,Unsupervised domain adaptation has attracted i...,NaN
6389,6390,Human Perception-based Evaluation Criterion fo...,2021,Computer vision technology is widely used in b...,"Neuroscience, Connectomics, Human perception, ..."
6390,6391,Improving OOD Generalization with Causal Invar...,2022,"In real-world applications, it is important an...","domain generalization, minimax problem, struct..."
6391,6392,Beyond Trivial Counterfactual Generations with...,2021,Explainability of machine learning models has ...,"Interpretability, Counterfactual, Explanations..."


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

#使用する学習済みモデル

#model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "microsoft/deberta-v3-base"
config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, config = config)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [7]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = PaperDataset(train_df, tokenizer)
val_dataset = PaperDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)

In [8]:
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [41]:
def train(model, dataloader, optimizer):
    model.train()
    train_loss = 0
    for batch in dataloader:
        
        #分類問題なので、目的変数の型をfloatからintに変更
        batch['targets'] = batch['targets'].type(torch.LongTensor)

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #targets = batch['targets'].view(-1, 1).to(device)

        # debaerta用
        targets = batch['targets'].to(device)
        train_correct = 0

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, labels=targets)

        # 損失の値
        loss = outputs.loss
        # それぞれのラベルの確率
        logits = outputs.logits
        # 確率の大きい方を取る
        _pred= logits.argmax(-1)
        pred = _pred.argmax(-1)

        # ラベルとpredが一致しているものの数だけtrain_correctを加算
        train_correct += pred.eq(targets).sum() 

        # if labels is not None:
        #     loss = nn.functional.binary_cross_entropy_with_logits(logits, labels) * labels.shape[1]
        # train_total += len(pred)
        
        #train_acc = train_correct/8 #正解率
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    return train_loss / len(dataloader)

In [42]:
def validate(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            batch['targets'] = batch['targets'].type(torch.LongTensor)

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            #targets = batch['targets'].view(-1, 1).to(device)

            # debaerta用
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask, labels=targets)

            loss = outputs.loss 

            val_loss += loss.item()

    return val_loss / len(dataloader)


In [43]:
df["y_pred"] = 0
for fold in df["folds"].unique():
    train_df = df[df["folds"] != fold]
    val_df = df[df["folds"] == fold]
    train_dataset = PaperDataset(train_df, tokenizer)
    val_dataset = PaperDataset(val_df, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)
    best_val_loss = float('inf')

    for epoch in tqdm(range(4)):
        train_loss = train(model, train_loader, optimizer)
        val_loss = validate(model, val_loader, optimizer)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
        torch.save(model.state_dict(), os.path.join(model_file_path,f"{model_checkpoint}_{fold}_model.pt")) # ベストなモデルを保存する
        print(f"{fold}_fold")
        #予測の実行
        outputs = predict(model, val_dataset)
        df.loc[val_df["y_pred"].index.to_list(),"y_pred"] = pd.Series(outputs)
        print("acc")
        print(accuracy_score(val_df["y"], pd.Series(outputs)))
        print(f'Epoch {epoch+1} - train loss: {train_loss:.3f}, val loss: {val_loss:.3f}')

  0%|          | 0/10 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 68.00 MiB (GPU 0; 15.78 GiB total capacity; 13.99 GiB already allocated; 52.69 MiB free; 14.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
#テスト

# モデルの読み込み
model.load_state_dict(torch.load(model_file_path))

test_dataset = PaperDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False,collate_fn=collate_fn)

In [70]:
def predict(model, dataloader):
    outputs = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # モデルによる予測
            output = model(input_ids, attention_mask=attention_mask)
            logits = output.logits
            # 確率の大きい方を取る
            _pred= logits.argmax(-1)
            #pred = _pred.argmax(-1)
            outputs.extend(_pred.tolist())
    return outputs

In [71]:
#予測の実行
outputs = predict(model, test_loader)

In [73]:
submit_df['y'] = pd.Series(outputs)
submit_df.to_csv(output_file_path,index = False)

# 検証用

In [14]:
#１バッチ(8データ)取り出す
for i in train_loader:
    batch = i
    break

In [21]:
batch['targets'].dtype

torch.float32

In [31]:
batch['targets'] = batch['targets'].type(torch.LongTensor)
batch['targets'].dtype

torch.int64

In [32]:
batch['targets']

tensor([1, 1, 0, 0, 0, 0, 0, 1])

In [33]:
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
targets = batch['targets'].to(device)


In [34]:
targets

tensor([1, 1, 0, 0, 0, 0, 0, 1], device='cuda:0')

In [35]:
label_index = (targets >= 0).nonzero()

In [36]:
label_index.view(-1)

tensor([0, 1, 2, 3, 4, 5, 6, 7], device='cuda:0')

In [44]:
outputs = model(input_ids, attention_mask, labels=targets)
outputs

SequenceClassifierOutput(loss=tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[0.0909, 0.1603],
        [0.1180, 0.1891],
        [0.0980, 0.1369],
        [0.0952, 0.1546],
        [0.1376, 0.2004],
        [0.1262, 0.1829],
        [0.1115, 0.1544],
        [0.0994, 0.1861]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [54]:
outputs = model(input_ids, attention_mask=attention_mask)
logits = outputs.logits
#print(logits)
# 確率の大きい方を取る
_pred= logits.argmax(-1)

print(_pred)



tensor([1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
